#August 2015 
#Author: FBB
#Modified by: L. Li (9/30/2015)

#reproduce the result in http://www.mdrc.org/sites/default/files/What%20Strategies%20Work%20for%20the%20Hard%20FR.pdf 


In [1]:
import os
import sys
import numpy as np
import pylab as pl
import json

%pylab inline

Populating the interactive namespace from numpy and matplotlib


#NULL HYPOTHESIS: the % of former prisoners convicted of a crime after release is the same or higher for candidates who participated in the program as for the control group, significance level p=0.05

#$H_0: P_0 - P_1 \leq$    0
    
#$H_a: P_0 - P_1 $> 0    
    
    
#$\alpha$ = 0.05    

##this is a TEST OF PROPORTIONS. we use the Binomial distribution since it is a yes/no (bernulli) test for each subject: the former inmate was or was not ever convicted of a crime in years 1-3 (second row in the Recidivism table):
#$P_0=0.488, P_1=0.431$

In [2]:
alpha=0.05
#we like fractions better then percentages. as a rule of thumb, either use fractions or counts
P_0=48.8*0.01 
P_1=43.1*0.01

n_0=409
n_1=568

#lets get the counts by multiplying by the sample size
Nt_0=P_0*n_0
Nt_1=P_1*n_1

print Nt_0, Nt_1

199.592 244.808


#2 samples, categorical data
#TWO OPTIONS z test, or chi square test.  

#START WITH Z TEST

##the z test compares the stanrard deviation of the expected distribution and the observed result. it tells you literally how many standard deviations from the tail an observation is, under the _assumption of normality
must define the sample standard deviation 

In [3]:
#define the sample proportion first
sp=(P_0*n_0+P_1*n_1)/(n_1+n_0)
print sp

0.454861821904


#standard deviation of the sampling distribution the distribution is Binomial, the binomail stdev is 

(see a proof here!: http://stats.stackexchange.com/questions/29641/standard-error-for-the-mean-of-a-sample-of-binomial-random-variables!): 

$\sqrt{\frac{p(1 - p)}{n}}$

for 2 samples this becomes 

$\sqrt{ \frac{ \hat{p}(1 - \hat{p})} {n1} + \frac{ \hat{p}(1 - \hat{p})} {n1} }$

cfr: page 138 of Statistics in a Nutshell, eq. 5.12 and here http://stattrek.com/hypothesis-test/difference-in-proportions.aspx?Tutorial=AP



Note that in the Image above, taken from the online version of the book, $\bar{x}$ should be $\hat{p}$!!

In [4]:
# FB: i am gonna create a little one line function to calculate the standard dev, it is not really needed, but just to show you how you do such a thing
sp_stdev= lambda p, n: np.sqrt( p * ( 1 - p ) /n[0] +  p * ( 1 - p )/n[1]  )

sp_stdev_2y=sp_stdev((Nt_0+Nt_1)/(n_0+n_1),[n_0,n_1])
print P_0, n_0, n_1, sp_stdev_2y

0.488 409 568 0.0322927107424


#z score: how many standard deviation away from the population parameter is my statistic?

#$z=\frac{P_1-P_0}{\sigma}$

In [5]:
zscore = lambda p0, p1, s : (p0-p1)/s
z_2y = zscore(P_1, P_0, sp_stdev_2y)
print z_2y


-1.76510421979


note that using p0-p1 or p1-p0 at the numerator is equivalent because the standardizes normal value of z has mean 0 (see image below) so that we can use the absolute value of the z score, or equivalently look for P[Z,z] if z is positive, and P[Z>z] if z is negative.

L. Li: rounding to -1.76 rather than -1.77 to get a conservative estimate of confidence, the corresponding alpha/p-value is 0.0392.

##if $p<\alpha$ : reject H0

##IMPORTANT!! note that this P is not the p-value, but 
##p-value=1-P

In [6]:
##p-value for crime conviction after 2 years
p_2y=0.0392


def report_result(p,a):
    print 'is the p value {0:.2f} smaller than the critical value {1:.2f}? '.format(p,a)
    if p<a:
        print "YES!"
    else: print "NO!"
    
    print 'the Null hypothesis is {}'.format( 'rejected' if p<a  else 'not rejected') 

    
report_result(p_2y,alpha)

is the p value 0.04 smaller than the critical value 0.05? 
YES!
the Null hypothesis is rejected


#Now lets do it with the $\chi^2$ test

#this can also be done with the $\chi^2$ distribution, see "Statistics In a Nutshell Chapter 4", or http://math.hws.edu/javamath/ryan/ChiSquare.html (if you are really just interested in the formula at face value)

##the chisq statistics tests a number against the distribution of the following quantity:

$$\chi^2 = \Sigma \frac{(observation - expectation)^2}{expectation}$$


if we talk about sample fractions  that is 

$$\chi^2 = \Sigma \frac{(f_{observed} - f_{expectated})^2}{f_{expected}}$$

##turns out this quantity is distributed according to a chi square distribution, so if i get the $\chi^2$ statistics i can compare it to the full chisq distribution and see how far in the tail it is

##the trickiest part (not that tricky) is to figure out the table of values. please see Statistics In a Nutshell Chapter 4, for our data for example:

|ever convicted of a crime (years 1-3) |     yes   | no   |                   
|---------------------------|----------------|------------------|---------------------------|
| test sample               | $43.1*5.68$    | $56.9*5.68$      | 568                       |
| control sample            | $48.8*4.09$     | $51.2*4.09$      | 409                       |
|                           |                |                  |                           |
| total                     | 444.4       |  532.6        | 977         |

the expected ratio is the product of the total of all rows and all columns, devided by the total

expected = $\frac{row tot * col tot}{total}$

In [7]:
Ntot = 977
expected = 568*409*444.4*532.6
sample_values = [[43.1*5.68,56.9*5.68],[48.8*4.09,51.2*4.09]]
 
chisqstat= lambda N, values, expect : N*((values[0][0]*values[1][1]-values[0][1]*values[1][0])**2)/(expect)

print chisqstat(Ntot,  sample_values, expected)

3.11559290673


3.12 is not larger then 3.84

FB: why am i mentioning 3.84?
L. Li: because 3.84 is the chi-squared test statistic for 1 degree of freedom (2 classes - 1) at alpha = 0.05

please  state what that means in terms of your hypothesis in a markdown cell below!

3.12 < 3.84, therefore, we cannot reject the null hypothesis at the 95% confidence level; however, 3.12 > 2.706 which is the threshold for the 90% confidence level. As consistent with the findings in the paper, the statistical significance level of 10% is upheld on an impact of the program on future crime convictions (program decreases probability of crime conviction).